In [1]:
import pandas as pd
import numpy as np
import os
import json
import re

os.chdir('./../')

In [2]:
# load the mapping dict to replace the original text with the cleaned text

clean_mapping = pd.read_excel('./data/clean_mapping.xlsx')
clean_mapping.fillna('', inplace=True)
mapping_dict = {}
for key, row in clean_mapping.iterrows():
  mapping_dict.update({row[0]: str(row[1])})


In [4]:
text_2_null = ''
mapping_dict2 = {}
for k, v in mapping_dict.items():
  if v=='':
    temp = '|' + k
    text_2_null += temp
  else:
    mapping_dict2.update({k:v})
mapping_dict2[text_2_null] = ''

In [5]:
df_train = pd.read_csv('./data/train.csv')
df_test = pd.read_csv('./data/test.csv')

In [ ]:
# this function is used to clean the tweets and remove unwanted characters and acronyms
def clean_tweet(tweet: str,
          mapping_dict: dict):
  for k, v in mapping_dict.items():
    tweet = re.sub(fr"{k}", v, tweet)
  tweet = re.sub(r"https?:\/\/t.co\/[A-Za-z0-9]+", "", tweet)

  punctuations = '@#!?+&*[]-%.:/();$=><|{}^' + "'`"
  for p in punctuations:
      tweet = tweet.replace(p, f' {p} ')

  # ... and ..
  tweet = tweet.replace('...', ' ... ')
  if '...' not in tweet:
      tweet = tweet.replace('..', ' ... ')

  # Acronyms
  tweet = re.sub(r"MH370", "Malaysia Airlines Flight 370", tweet)
  tweet = re.sub(r"mÌ¼sica", "music", tweet)
  tweet = re.sub(r"okwx", "Oklahoma City Weather", tweet)
  tweet = re.sub(r"arwx", "Arkansas Weather", tweet)
  tweet = re.sub(r"gawx", "Georgia Weather", tweet)
  tweet = re.sub(r"scwx", "South Carolina Weather", tweet)
  tweet = re.sub(r"cawx", "California Weather", tweet)
  tweet = re.sub(r"tnwx", "Tennessee Weather", tweet)
  tweet = re.sub(r"azwx", "Arizona Weather", tweet)
  tweet = re.sub(r"alwx", "Alabama Weather", tweet)
  tweet = re.sub(r"wordpressdotcom", "wordpress", tweet)
  tweet = re.sub(r"usNWSgov", "United States National Weather Service", tweet)
  tweet = re.sub(r"Suruc", "Sanliurfa", tweet)

  # Grouping same words without embeddings
  tweet = re.sub(r"Bestnaijamade", "bestnaijamade", tweet)
  tweet = re.sub(r"SOUDELOR", "Soudelor", tweet)

  return tweet

In [ ]:
df_train['text_cleaned'] = df_train['text'].apply(lambda s : clean_tweet(s, mapping_dict2))
df_test['text_cleaned'] = df_test['text'].apply(lambda s : clean_tweet(s, mapping_dict2))

In [ ]:
df_train['target_relabeled'] = df_train['target'].copy()

df_train.loc[df_train['text'] == 'like for the music video I want some real action shit like burning buildings and police chases not some weak ben winston shit', 'target_relabeled'] = 0
df_train.loc[df_train['text'] == 'Hellfire is surrounded by desires so be careful and donÛªt let your desires control you! #Afterlife', 'target_relabeled'] = 0
df_train.loc[df_train['text'] == 'To fight bioterrorism sir.', 'target_relabeled'] = 0
df_train.loc[df_train['text'] == '.POTUS #StrategicPatience is a strategy for #Genocide; refugees; IDP Internally displaced people; horror; etc. https://t.co/rqWuoy1fm4', 'target_relabeled'] = 1
df_train.loc[df_train['text'] == 'CLEARED:incident with injury:I-495  inner loop Exit 31 - MD 97/Georgia Ave Silver Spring', 'target_relabeled'] = 1
df_train.loc[df_train['text'] == '#foodscare #offers2go #NestleIndia slips into loss after #Magginoodle #ban unsafe and hazardous for #humanconsumption', 'target_relabeled'] = 0
df_train.loc[df_train['text'] == 'In #islam saving a person is equal in reward to saving all humans! Islam is the opposite of terrorism!', 'target_relabeled'] = 0
df_train.loc[df_train['text'] == 'Who is bringing the tornadoes and floods. Who is bringing the climate change. God is after America He is plaguing her\n \n#FARRAKHAN #QUOTE', 'target_relabeled'] = 1
df_train.loc[df_train['text'] == 'RT NotExplained: The only known image of infamous hijacker D.B. Cooper. http://t.co/JlzK2HdeTG', 'target_relabeled'] = 1
df_train.loc[df_train['text'] == "Mmmmmm I'm burning.... I'm burning buildings I'm building.... Oooooohhhh oooh ooh...", 'target_relabeled'] = 0
df_train.loc[df_train['text'] == "wowo--=== 12000 Nigerian refugees repatriated from Cameroon", 'target_relabeled'] = 0
df_train.loc[df_train['text'] == "He came to a land which was engulfed in tribal war and turned it into a land of peace i.e. Madinah. #ProphetMuhammad #islam", 'target_relabeled'] = 0
df_train.loc[df_train['text'] == "Hellfire! We donÛªt even want to think about it or mention it so letÛªs not do anything that leads to it #islam!", 'target_relabeled'] = 0
df_train.loc[df_train['text'] == "The Prophet (peace be upon him) said 'Save yourself from Hellfire even if it is by giving half a date in charity.'", 'target_relabeled'] = 0
df_train.loc[df_train['text'] == "Caution: breathing may be hazardous to your health.", 'target_relabeled'] = 1
df_train.loc[df_train['text'] == "I Pledge Allegiance To The P.O.P.E. And The Burning Buildings of Epic City. ??????", 'target_relabeled'] = 0
df_train.loc[df_train['text'] == "#Allah describes piling up #wealth thinking it would last #forever as the description of the people of #Hellfire in Surah Humaza. #Reflect", 'target_relabeled'] = 0
df_train.loc[df_train['text'] == "that horrible sinking feeling when youÛªve been at home on your phone for a while and you realise its been on 3G this whole time", 'target_relabeled'] = 0

In [ ]:
df_train.to_excel('./data/df_train.xlsx', index=False)
df_test.to_excel('./data/df_test.xlsx', index=False)